<a href="https://colab.research.google.com/github/sanvasu/Netflix---Data-Exploration-and-Visualisation/blob/main/Netflix_Casestudy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv('netflix.csv')

In [3]:
df = pd.DataFrame(data)

In [4]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


**BASIC ANALYSIS**
1. Un-nesting the columns,
    *In casr,listed_in_country and director columns there are same column with multiple comma seperated values.
    So used the str.split function to un-nest and used explode method to convert each element of a column into multiple rows.*


In [5]:
df['cast'] = df['cast'].str.split(',')
df['listed_in'] = df['listed_in'].str.split(',')
df['country'] = df['country'].str.split(',')
df['director'] = df['director'].str.split(',')

In [6]:
df = df.explode('cast').explode('listed_in').explode('country').explode('director').reset_index(drop=True)

**BASIC ANALYSIS **

Used df.info() method to find the datatype of each column

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202065 entries, 0 to 202064
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   show_id       202065 non-null  object
 1   type          202065 non-null  object
 2   title         202065 non-null  object
 3   director      151422 non-null  object
 4   cast          199916 non-null  object
 5   country       190168 non-null  object
 6   date_added    201907 non-null  object
 7   release_year  202065 non-null  int64 
 8   rating        201998 non-null  object
 9   duration      202062 non-null  object
 10  listed_in     202065 non-null  object
 11  description   202065 non-null  object
dtypes: int64(1), object(11)
memory usage: 18.5+ MB


In [8]:
df_obj = df.select_dtypes('object')

Used strip method to remove the trailing whtespaces.

In [9]:
df[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

In [10]:
df.isna().sum()

,0
show_id,0
type,0
title,0
director,50643
cast,2149
country,11897
date_added,158
release_year,0
rating,67
duration,3


After removed the white space, we can find some columns have null values. So inorder to replace the null values, used replace function for the selected columns.

In [11]:
df['director'] = df['director'].replace(np.nan, 'Unknown director')
df['cast'] = df['cast'].replace(np.nan, 'Unknown cast')
df['country'] = df['country'].replace(np.nan, 'Unknown country')
df['date_added'] = df['date_added'].replace(np.nan, 'Unknown date')
df['rating'] = df['rating'].replace(np.nan, 'Unknown rating')
df['duration'] = df['duration'].replace(np.nan, 'Unknown duration')

In [12]:
df.isna().sum()

,0
show_id,0
type,0
title,0
director,0
cast,0
country,0
date_added,0
release_year,0
rating,0
duration,0


Now, the data is all clear without null and any whitespaces.

In [13]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,Unknown cast,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,Unknown director,Ama Qamata,South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,International TV Shows,"After crossing paths at a party, a Cape Town t..."
2,s2,TV Show,Blood & Water,Unknown director,Ama Qamata,South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,TV Dramas,"After crossing paths at a party, a Cape Town t..."
3,s2,TV Show,Blood & Water,Unknown director,Ama Qamata,South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,TV Mysteries,"After crossing paths at a party, a Cape Town t..."
4,s2,TV Show,Blood & Water,Unknown director,Khosi Ngema,South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,International TV Shows,"After crossing paths at a party, a Cape Town t..."


Finding categorical variable both using graphical and non-graphical analysis.

In a Non-Graphical Analysis:

In [14]:
category_coumn = ['show_id','type','title','director','cast','country','date_added','rating','duration','listed_in']
for column in category_coumn:
    print("Value counts for column '{}':".format(column))
    print(df[column].value_counts())
    print("\n")

Value counts for column 'show_id':
show_id
s7165    700
s6985    504
s7516    468
s2554    416
s5306    378
        ... 
s6330      1
s8176      1
s937       1
s3387      1
s1         1
Name: count, Length: 8807, dtype: int64


Value counts for column 'type':
type
Movie      145917
TV Show     56148
Name: count, dtype: int64


Value counts for column 'title':
title
Kahlil Gibran's The Prophet    700
Holidays                       504
Movie 43                       468
The Eddy                       416
Narcos                         378
                              ... 
Blackfish                        1
The 2000s                        1
Miniforce: Super Dino Power      1
Dancing with the Birds           1
Dick Johnson Is Dead             1
Name: count, Length: 8806, dtype: int64


Value counts for column 'director':
director
Unknown director       50643
Martin Scorsese          419
Youssef Chahine          409
Cathy Garcia-Molina      356
Steven Spielberg         355
               